In [1]:
import numpy as np
import warnings
from pandas_datareader import data as pdr
import yfinance as yf
import datetime as dt
from yahoo_fin import stock_info as si
import pandas as pd
pd.set_option('display.max_rows', None)
warnings.filterwarnings("ignore")
yf.pdr_override()

In [2]:
num_of_years = 1
start = dt.date.today() - dt.timedelta(days = int(365.25*num_of_years))
end = dt.date.today()

In [ ]:
# df = pd.read_csv('data/mktcap.csv', index_col=0)
# df = df[df['mktcap']>0]
#
# Use csv file from China
df = pd.read_csv('data/hk_stock_full.csv', header=None)
df = df.rename(columns={0: 'stock'})

In [ ]:
df

In [ ]:
def stock_begin_timegap(stock):
    num_of_years = 1
    start = dt.date.today() - dt.timedelta(days = int(365.25*num_of_years))
    try:
        stock_start = yf.download(stock, start=start).index[0]
        return (stock_start - pd.to_datetime(start)).days
    except:
        return 10000

In [ ]:
df['gap'] = df['stock'].apply(stock_begin_timegap)

In [ ]:
df

In [ ]:
stock_df = df[df['gap'] <= 5]

In [ ]:
stock_df.count()

In [ ]:
stock_df.to_csv('data/stock_qualified.csv')

### Find correlation

In [3]:
stock_df = pd.read_csv('data/stock_qualified.csv', index_col=0)

In [4]:
%%time
tickers = stock_df['stock'].to_list()
dataset = pdr.get_data_yahoo(tickers, start, end)['Adj Close']

[*********************100%***********************]  2171 of 2171 completed
CPU times: user 2min 19s, sys: 18.3 s, total: 2min 37s
Wall time: 3min 46s


In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 256 entries, 2019-11-18 to 2020-11-16
Columns: 2171 entries, 0001.HK to 9900.HK
dtypes: float64(2171)
memory usage: 4.2 MB


In [7]:
dataset.to_csv('data/all_stock_close.csv')

In [8]:
dataset.head(3)

0001.HK    0002.HK    0003.HK    0004.HK    0005.HK    0006.HK  \
Date                                                                           
2019-11-18  68.200218  76.621101  13.894835  18.031931  57.980183  51.817047   
2019-11-19  69.289536  77.142654  14.061546  18.130037  58.627724  52.288540   
2019-11-20  69.052727  76.621101  14.006008  17.816097  58.279049  52.524284   

            0007.HK   0008.HK  0009.HK    0010.HK  ...  8612.HK  8613.HK  \
Date                                               ...                     
2019-11-18     0.86  4.268155    0.122  19.018490  ...     0.56    0.070   
2019-11-19     0.86  4.324192    0.119  19.213554  ...     0.53    0.068   
2019-11-20     0.84  4.333531    0.118  19.330589  ...     0.53    0.068   

            8619.HK  8621.HK  8622.HK  8627.HK  8631.HK   8635.HK  8668.HK  \
Date                                                                         
2019-11-18     1.74     0.49    0.139     0.24     0.16  0.118341    0.162   
2019-11-19     1.70     0.56    0.126     0.51     0.16  0.121275    0.162   
2019-11-20     1.68     0.69    0.127     0.46     0.17  0.124209    0.165   

            9900.HK  
Date                 
2019-11-18     1.45  
2019-11-19     1.47  
2019-11-20     1.42  

[3 rows x 2171 columns]

In [ ]:
dataset['']

### Use close on all stocks to compile result

In [3]:
dataset = pd.read_csv('data/all_stock_close.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
dataset.set_index('Date', drop=True, inplace=True)

In [4]:
prevday_returns = np.log(dataset.shift(1)/dataset.shift(2))
today_returns = np.log(dataset/dataset.shift(1))

In [5]:
stocks_prev = prevday_returns.columns.to_list()
stocks_today = today_returns.columns.to_list()

In [6]:
new_col = ['{}.prev'.format(c) for c in stocks_prev]
prevday_returns.columns = new_col

In [7]:
comb_df = pd.concat([prevday_returns, today_returns], axis=1)

In [8]:
%%time
print('\nCorrelation Matrix')
corr_matrix = comb_df.corr()
# print(corr_matrix)


Correlation Matrix
CPU times: user 8.59 s, sys: 83.4 ms, total: 8.67 s
Wall time: 8.66 s


In [9]:
corr_matrix_corrected = corr_matrix.loc[new_col, stocks_today]

### What to drop

In [17]:
sorted_corr = corr_matrix_corrected.abs().unstack().sort_values(ascending=False)
sorted_corr.shape

(4713241,)

In [18]:
sorted_corr.head(10)

8612.HK  6898.HK.prev    0.796597
1121.HK  1195.HK.prev    0.778497
1315.HK  0056.HK.prev    0.769216
1636.HK  8631.HK.prev    0.755231
6898.HK  1216.HK.prev    0.752972
1216.HK  1371.HK.prev    0.690528
8427.HK  0508.HK.prev    0.675625
1371.HK  1143.HK.prev    0.673683
1003.HK  0612.HK.prev    0.670347
1439.HK  8627.HK.prev    0.667464
dtype: float64